In [1]:
# PoC using scaning and spending keys

In [2]:
import hashlib
from py_ecc.secp256k1 import *
import sha3
from eth_account import Account

## Sender

$S = G*s$

In [3]:
# privkey: 0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30
# address: 0x3CB39EA2f14B16B69B451719A7BEd55e0aFEcE8F
s = int(0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30) # private key
S = secp256k1.privtopub(s.to_bytes(32, "big")) # public key
S

(22246744184454969143801186698733154500632648736073949898323976612504587645286,
 110772761940586493986212935445517909380300793379795289150161960681985511655321)

## Recipient

$P = G*p$

In [4]:
# privkey: 0x0000000000000000000000000000000000000000000000000000000000000001
# address: 0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf
p_scan = int(0x0000000000000000000000000000000000000000000000000000000000000002) # private key
p_spend = int(0x0000000000000000000000000000000000000000000000000000000000000003) # private key

P_scan = secp256k1.privtopub(p_scan.to_bytes(32, "big")) # public key
P_spend = secp256k1.privtopub(p_spend.to_bytes(32, "big")) # public key
P_scan, P_spend

((89565891926547004231252920425935692360644145829622209833684329913297188986597,
  12158399299693830322967808612713398636155367887041628176798871954788371653930),
 (112711660439710606056748659173929673102114977341539408544630613555209775888121,
  25583027980570883691656905877401976406448868254816295069919888960541586679410))

## Calculate Stealth Address: $P_{spend} + G*hash(Q)$

$Q = S * p_{scan}$

In [5]:
Q = secp256k1.multiply(P_scan, s)
Q

(65311808848028536848162101908966111079795231803322390815513763038079235257196,
 43767810034999830518515787564234053904327508763526333662117780420755425490082)

$Q = S * p_{scan} = P_{scan} * s$

In [6]:
assert Q == secp256k1.multiply(S, p_scan)

$h(Q)$

In [7]:
Q_hex = sha3.keccak_256(Q[0].to_bytes(32, "big") 
                        + Q[1].to_bytes(32, "big")
                       ).hexdigest()
Q_hased = bytearray.fromhex(Q_hex)

$ stA = h(Q) * G + P_{spend}$

#### Sender sends funds to...

In [8]:
stP = secp256k1.add(P_spend, secp256k1.privtopub(Q_hased))
stA = "0x"+ sha3.keccak_256(stP[0].to_bytes(32, "big")
                            +stP[1].to_bytes(32, "big")
                           ).hexdigest()[-40:]
stA

'0xfed69df0a27f1dae0d7430ead82aaedfad6332bb'

#### Sender broadcasts

In [9]:
S, stA

((22246744184454969143801186698733154500632648736073949898323976612504587645286,
  110772761940586493986212935445517909380300793379795289150161960681985511655321),
 '0xfed69df0a27f1dae0d7430ead82aaedfad6332bb')

## Parse received funds

* Note that $p_{scan}$ and $P_{spend}$ can be shared with a trusted party
* There may be many S to be parsed

$h(p_{scan}*S)*G + P_{spend} => toAddress$

In [10]:
Q = secp256k1.multiply(S, p_scan)
Q_hex = sha3.keccak_256(Q[0].to_bytes(32, "big")+Q[1].to_bytes(32, "big")).hexdigest()
Q_hased = bytearray.fromhex(Q_hex)

P_stealth = secp256k1.add(P_spend, secp256k1.privtopub(Q_hased))
P_stealthAddress  = "0x"+ sha3.keccak_256(stP[0].to_bytes(32, "big")
                                        + stP[1].to_bytes(32, "big")
                                        ).hexdigest()[-40:]
P_stealthAddress

'0xfed69df0a27f1dae0d7430ead82aaedfad6332bb'

logged stealth address $stA$ equals the derived stealth address $P_stealthAddress$

$stA==stA_d$

In [11]:
P_stealthAddress == stA

True

## Derive private key

#### Only the recipient has access to $p_{spend}$

$p_{stealth}=p_{spend}+hash(Q)$

In [12]:
Q = secp256k1.multiply(S, p_scan)
Q_hex = sha3.keccak_256(Q[0].to_bytes(32, "big")+Q[1].to_bytes(32, "big")).hexdigest()
p_stealth = p_spend + int(Q_hex, 16)
p_stealth

39153944482575822531387237249775711740128993925789544779866399859639729033274

$P_{stealth} = p_{stealth}*G$

In [13]:
# Recipient has private key to ...
P_stealth = secp256k1.privtopub(p_stealth.to_bytes(32, "big"))
P_stealth

(67663851387124608323744162645277269585638670865381831245083336172545348387042,
 80449904826544093817252981338261706033086352950841917067356875711772573870404)

In [14]:
P_stealthAddress_d  = "0x"+ sha3.keccak_256(P_stealth[0].to_bytes(32, "big")
                                        + P_stealth[1].to_bytes(32, "big")
                                        ).hexdigest()[-40:]
P_stealthAddress_d

'0xfed69df0a27f1dae0d7430ead82aaedfad6332bb'

In [15]:
Account.from_key((p_stealth).to_bytes(32, "big")).address

'0xfEd69Df0a27F1daE0D7430EAd82aaEdfAD6332bb'

## Additionally add view tags

In addition to S and stA, the sender also broadcasts the first byte of h(Q)

In [16]:
Q_hased[0]

86

The recipient can do the the same a before without one EC Multiplication, one EC Addition and on Public Key to Address Conversion in order to check being a potential recipient.

In [17]:
Q_derived = secp256k1.multiply(S, p_scan)
Q_hex_derived = sha3.keccak_256(Q_derived[0].to_bytes(32, "big")
                                +Q_derived[1].to_bytes(32, "big")
                               ).hexdigest()
Q_hashed_derived = bytearray.fromhex(Q_hex_derived)

Check view tag

In [18]:
run = Q_hased[0] == Q_hashed_derived[0] 
run

True

In [19]:
if run:
    P_stealth = secp256k1.add(P_spend, secp256k1.privtopub(Q_hased))
    P_stealthAddress  = "0x"+ sha3.keccak_256(stP[0].to_bytes(32, "big")
                                            + stP[1].to_bytes(32, "big")
                                            ).hexdigest()[-40:]
P_stealthAddress

'0xfed69df0a27f1dae0d7430ead82aaedfad6332bb'

In [20]:
P_stealthAddress==stA

True